SQLite does not support stored procedures like MySQL, but we can achieve similar functionality using Python functions in a Jupyter Notebook with Pandas and SQLite.

Alters the collections table to add a deleted column.

Defines current_collections() to fetch non-deleted artwork.

Implements sell(sold_title) to:

Check if the artwork exists and is not deleted.

Mark it as sold by setting deleted = 1.

Log the sale in the transactions table.

Prevent selling an already sold artwork.

Displays data before and after selling to show the changes.

Expected Output
Before Selling: The artwork appears in collections.

After Selling: The artwork is marked as deleted and appears in transactions.

In [1]:
import sqlite3
import shutil
import pandas as pd

# Paths for original and copy databases
original_db = "../Datasets/database/mfa.db"
copy_db = "mfa_copy.db"

# Make a copy of the database
shutil.copy(original_db, copy_db)

# Connect to the copied database
conn = sqlite3.connect(copy_db)
cursor = conn.cursor()

# Add 'deleted' column if not exists
cursor.execute("""
ALTER TABLE collections 
ADD COLUMN deleted INTEGER DEFAULT 0;
""")

# Create transactions table if not exists
cursor.execute("""
CREATE TABLE IF NOT EXISTS transactions (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    title TEXT NOT NULL,
    action TEXT CHECK(action IN ('bought', 'sold')) NOT NULL
);
""")

conn.commit()

# Function to get non-deleted collections
def current_collections():
    query = "SELECT title, accession_number, acquired FROM collections WHERE deleted = 0"
    return pd.read_sql_query(query, conn)

# Function to mark an artwork as sold
def sell(sold_title):
    # Check if item exists and is not deleted
    cursor.execute("SELECT id, title FROM collections WHERE title = ? AND deleted = 0", (sold_title,))
    result = cursor.fetchone()
    
    if result:
        sold_id, title = result
        cursor.execute("UPDATE collections SET deleted = 1 WHERE id = ?", (sold_id,))
        cursor.execute("INSERT INTO transactions (title, action) VALUES (?, 'sold')", (title,))
        conn.commit()
        print(f"Artwork '{title}' marked as sold.")
    else:
        print(f"Error: Artwork '{sold_title}' not available for sale.")

# Display current collections before selling
print("Collections Before Selling:")
display(current_collections())

# Sell an artwork
sell('Farmers working at dawn')

# Display collections after selling
print("Collections After Selling:")
display(current_collections())

# Display transactions
print("Transaction Log:")
display(pd.read_sql_query("SELECT * FROM transactions", conn))

# Close the connection
conn.close()


Collections Before Selling:


,title,accession_number,acquired
0,Farmers working at dawn,11.6152,1911-08-03
1,Imaginative landscape,56.496,None
2,Profusion of flowers,56.257,1956-04-12
3,Spring outing,14.76,1914-01-08


Artwork 'Farmers working at dawn' marked as sold.
Collections After Selling:


,title,accession_number,acquired
0,Imaginative landscape,56.496,None
1,Profusion of flowers,56.257,1956-04-12
2,Spring outing,14.76,1914-01-08


Transaction Log:


,id,title,action
0,1,Farmers working at dawn,sold
